In [1]:
#installation de Chromium
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb
!sudo apt-get update
!sudo -E apt-get install -y /tmp/chrome.deb
!pip install chromedriver-autoinstaller selenium

import chromedriver_autoinstaller
path_to_web_driver = chromedriver_autoinstaller.install()

--2025-12-15 19:46:43--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 74.125.206.136, 74.125.206.93, 74.125.206.190, ...
Connecting to dl.google.com (dl.google.com)|74.125.206.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117886796 (112M) [application/x-debian-package]
Saving to: ‘/tmp/chrome.deb’

/tmp/chrome.deb     100%[===================>] 112.42M  99.9MB/s    in 1.1s    

2025-12-15 19:46:44 (99.9 MB/s) - ‘/tmp/chrome.deb’ saved [117886796/117886796]

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu noble InRelease
Get:3 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]        
Get:4 http://archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]      
Get:5 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Get:6 http://archive.ubuntu.com/ubuntu noble-updates/main amd64 Packages [2,124 kB]

In [2]:
#import de modules et initialisation du webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--verbose')

In [20]:
#lancement du navigateur Chrome
service = Service(executable_path=path_to_web_driver)

browser = webdriver.Chrome(
    service=service,
    options=chrome_options
)

In [21]:
#chargement de la page airparif
browser.get('https://www.airparif.fr/surveiller-la-pollution/bilans-et-cartes-annuels-de-pollution')
time.sleep(10)

In [22]:
#choix de l'année
année="2019"
from selenium.webdriver.support.ui import Select
wait = WebDriverWait(browser, 10)
element_menu = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="date-filter"]')))
menu_deroulant = Select(element_menu)
menu_deroulant.select_by_visible_text(année)

time.sleep(3)

In [24]:
#localisation de la barre des éléments : attention rajouter une intitialisation forcée à pm10
from selenium.common.exceptions import StaleElementReferenceException

#l=["Palaiseau","Paris 11","Paris 13", "Melun", "Versailles"]

def fonction_taux_de_particule(l):
    search = browser.find_element("xpath",'//*[@id="carte-annuelle-panel-info"]//input')
    wait = WebDriverWait(browser, 3, ignored_exceptions=(StaleElementReferenceException,))
    valeur=[{"Commune" : e ,"pm10":None , "pm25":None , "no2":None} for e in l]
    liste_polluant=["pm10","pm25","no2"]
    bouton_polluant = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[data-pid={liste_polluant[0]}]")))
    browser.execute_script("arguments[0].click();", bouton_polluant)
    print(f"Clic forcé sur le {liste_polluant[0]} effectué via JavaScript.")
    time.sleep(5)
    for j in range(len(liste_polluant)):
        #Remarque:initialement on se situe sur le pm10 par défaut
        polluant=liste_polluant[j]
        if j <=1:
            polluant_suivant=liste_polluant[j+1]
        else:
            polluant_suivant=None
        for i in range(len(l)):
            #1/la barre de recherche
            search.clear()
            search.send_keys(l[i])  
            #2/la barre de suggestion
            try:
                suggestion_a_cliquer= wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"pac-item")))
            #if (i%50==0):
            #    print(suggestion_a_cliquer.text)
                suggestion_a_cliquer.click()
                time.sleep(0.2)
            #3/le taux de particules dans l'air
                try:
                    taux_de_particules=wait.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="current-params"]/strong')))
                    valeur[i][polluant]=taux_de_particules.text
                except Exception as e:
                    print(f"Une erreur 1 s'est produite pour la commune {l[i]} et le polluant {liste_polluant[j]} -> attribution None aux valeurs")
            except Exception as e:
                print(f"Une erreur 2 s'est produite pour la commune {l[i]} et le polluant {liste_polluant[j]} -> attribution None aux valeurs")
        #chargement page nouveau polluant
        #print(f"polluant: {polluant} done")
        if polluant_suivant !=None:
            bouton_polluant = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[data-pid={polluant_suivant}]")))
            browser.execute_script("arguments[0].click();", bouton_polluant)
            print(f"Clic forcé sur le {polluant_suivant} effectué via JavaScript.")
            time.sleep(5)
    #df=pd.DataFrame(valeur)    
    #return(df)
    return(valeur)

#df=fonction_taux_de_particule(l)
#print(df)
        

In [25]:
df=pd.read_csv("/home/onyxia/work/projet_python_data_science/donnees_niveau_de_vie.csv")
liste_commune=list(df["nom_commune_postal"])
print(liste_commune)


['ABBEVILLE LA RIVIERE', 'ABLEIGES', 'ABLIS', 'ABLON SUR SEINE', 'ACHERES', 'ACHERES LA FORET', 'ADAINVILLE', 'AIGREMONT', 'AINCOURT', 'ALFORTVILLE', 'ALLAINVILLE', 'AMBLEVILLE', 'AMENUCOURT', 'AMILLIS', 'AMPONVILLE', 'ANDELU', 'ANDILLY', 'ANDRESY', 'ANDREZEL', 'ANGERVILLE', 'ANGERVILLIERS', 'ANNET SUR MARNE', 'ANTONY', 'ARBONNE LA FORET', 'ARCUEIL', 'ARGENTEUIL', 'ARGENTIERES', 'ARMENTIERES EN BRIE', 'ARNOUVILLE', 'ARNOUVILLE LES MANTES', 'ARPAJON', 'ARRANCOURT', 'ARRONVILLE', 'ARTHIES', 'ARVILLE', 'ASNIERES SUR OISE', 'ASNIERES SUR SEINE', 'ATHIS MONS', 'ATTAINVILLE', 'AUBEPIERRE OZOUER LE REPOS', 'AUBERGENVILLE', 'AUBERVILLIERS', 'AUFFARGIS', 'AUFFERVILLE', 'AUFFREVILLE BRASSEUIL', 'AUGERS EN BRIE', 'AULNAY SOUS BOIS', 'AULNAY SUR MAULDRE', 'AULNOY', 'AUTEUIL', 'AUTHON LA PLAINE', 'AUTOUILLET', 'AUVERNAUX', 'AUVERS ST GEORGES', 'AUVERS SUR OISE', 'AVERNES', 'AVON', 'AVRAINVILLE', 'BABY', 'BAGNEAUX SUR LOING', 'BAGNEUX', 'BAGNOLET', 'BAILLET EN FRANCE', 'BAILLY', 'BAILLY ROMAINVILLIE

In [ ]:
#extraction par paquet de 50
def extraction_donnee(l):
    nombre2paquet=len(l)//50
    reste=len(l)-nombre2paquet
    decoupage=[(i+1)*50 for i in range(nombre2paquet)]
    if reste!=0:
        decoupage.append(reste)
    inf=0
    valeur=[]
    for j in decoupage:
        paquet=l[inf:j]
        v=fonction_taux_de_particule(paquet)
        if type(v)==list:
            valeur.extend(v)
            print(f"paquet {inf} à {j} traité -> paquet suivant")
            df=pd.DataFrame(valeur) 
            df.to_csv(f"donnees_commune_pollution_paquet{j}.csv",index=False)
        else:
            print(f"paquet {inf} à {j} non traité : erreur -> paquet suivant")
        inf=j
    return(pd.DataFrame(valeur))
    
    
        
df2=extraction_donnee(liste_commune)

Clic forcé sur le pm10 effectué via JavaScript.
Clic forcé sur le pm25 effectué via JavaScript.
Clic forcé sur le no2 effectué via JavaScript.
paquet 0 à 50 traité -> paquet suivant
Clic forcé sur le pm10 effectué via JavaScript.
Clic forcé sur le pm25 effectué via JavaScript.
Clic forcé sur le no2 effectué via JavaScript.
paquet 50 à 100 traité -> paquet suivant
Clic forcé sur le pm10 effectué via JavaScript.
Clic forcé sur le pm25 effectué via JavaScript.
Clic forcé sur le no2 effectué via JavaScript.
paquet 100 à 150 traité -> paquet suivant
Clic forcé sur le pm10 effectué via JavaScript.
Clic forcé sur le pm25 effectué via JavaScript.
Clic forcé sur le no2 effectué via JavaScript.
paquet 150 à 200 traité -> paquet suivant
Clic forcé sur le pm10 effectué via JavaScript.
Une erreur 1 s'est produite pour la commune CHARMONT et le polluant pm10 -> attribution None aux valeurs
Une erreur 1 s'est produite pour la commune CHAUFFRY et le polluant pm10 -> attribution None aux valeurs
Clic f

In [ ]:
polluant_suivant="pm25"
bouton_pm25 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[data-pid={polluant_suivant}]")))
# 2. On clique dessus
browser.execute_script("arguments[0].click();", bouton_pm25)
print("Clic forcé sur PM2.5 effectué via JavaScript.")

Clic forcé sur PM2.5 effectué via JavaScript.
